# DL-TFM in Pytorch I

## Imports and technical prerequisites

In [56]:
from tracNet import TracNet

import copy
import datetime
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Tuple
from scipy.io import loadmat, savemat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

%matplotlib inline

Set seeds for reproducability.

In [57]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False

Use CUDA if available.

In [58]:
gc.collect()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Running on device: {device}")

Running on device: cuda


## 1. Data loading and preprocessing

Set paths to training and test data.

In [59]:
# Training data
dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/dspl'
dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/dsplRadial'
trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/trac'
tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/train/trainData104/tracRadial'

# Test data
test_dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_dspl'
test_trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/Repos/DL-TFM-main/test/generic/testData104/mini_trac'

In [60]:
def data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path, test_dspl_path, test_trac_path):
    # number_samples = len([name for name in os.listdir(dspl_path) if os.path.isfile(os.path.join(dspl_path, name))])
    # number_radials = len([name for name in os.listdir(dsplRadial_path) if os.path.isfile(os.path.join(dsplRadial_path, name))])
    
    # save all samples in matrix
    samples = [] 
    for i, filename in enumerate(os.listdir(dspl_path)):
        f = os.path.join(dspl_path, filename)
        if os.path.isfile(f):
            sample = loadmat(f)
            if '__header__' in sample: del sample['__header__']
            if '__version__' in sample: del sample['__version__']
            if '__globals__' in sample: del sample['__globals__']
            sample['name'] = filename
            samples = np.append(samples, sample)
        else:
            continue
    samples = np.array(samples)

    # save all radial patterns of displacements in matrix
    dspl_radials = []
    for i, filename in enumerate(os.listdir(dsplRadial_path)):
        f = os.path.join(dsplRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            dspl_radials = np.append(dspl_radials, radial)
        else:
            continue
    dspl_radials = np.array(dspl_radials)
    
    # save all targets in matrix
    targets = []
    for i, filename in enumerate(os.listdir(trac_path)):
        f = os.path.join(trac_path, filename)
        if os.path.isfile(f):
            target = loadmat(f)
            if '__header__' in target: del target['__header__']
            if '__version__' in target: del target['__version__']
            if '__globals__' in target: del target['__globals__']
            target['name'] = filename
            targets = np.append(targets, target)
        else:
            continue 
    targets = np.array(targets)
    
    # save all radial patterns of traction forces in matrix
    trac_radials = []
    for i, filename in enumerate(os.listdir(tracRadial_path)):
        f = os.path.join(tracRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            trac_radials = np.append(trac_radials, radial)
        else:
            continue
    trac_radials = np.array(trac_radials)
    
    # save test displacements in matrix
    test_dspls = [] 
    for i, filename in enumerate(os.listdir(test_dspl_path)):
        f = os.path.join(test_dspl_path, filename)
        if os.path.isfile(f):
            test_dspl = loadmat(f)
            if '__header__' in test_dspl: del test_dspl['__header__']
            if '__version__' in test_dspl: del test_dspl['__version__']
            if '__globals__' in test_dspl: del test_dspl['__globals__']
            test_dspl['name'] = filename
            test_dspls = np.append(test_dspls, test_dspl)
        else:
            continue
    test_dspls = np.array(test_dspls)
    
    # save test traction fields in matrix
    test_tracs = [] 
    for i, filename in enumerate(os.listdir(test_trac_path)):
        f = os.path.join(test_trac_path, filename)
        if os.path.isfile(f):
            test_trac = loadmat(f)
            if '__header__' in test_trac: del test_trac['__header__']
            if '__version__' in test_trac: del test_trac['__version__']
            if '__globals__' in test_trac: del test_trac['__globals__']
            test_trac['name'] = filename
            test_tracs = np.append(test_tracs, test_trac)
        else:
            continue
    test_tracs = np.array(test_tracs)
    
    return samples, dspl_radials, targets, trac_radials, test_dspls, test_tracs

In [61]:
def matFiles_to_npArray(path):

    instances = [] 
    for i, filename in enumerate(os.listdir(path)):
        f = os.path.join(path, filename)
        if os.path.isfile(f):
            instance = loadmat(f)
            if '__header__' in instance: del instance['__header__']
            if '__version__' in instance: del instance['__version__']
            if '__globals__' in instance: del instance['__globals__']
            instance['name'] = filename
            instances = np.append(instances, instance)
        else:
            continue
    
    return np.array(instances)

Create numpy arrays for samples and targets.

In [62]:
samples = matFiles_to_npArray(dspl_path) 
dspl_radials = matFiles_to_npArray(dsplRadial_path)
targets = matFiles_to_npArray(trac_path)
trac_radials = matFiles_to_npArray(tracRadial_path)
test_dspls = matFiles_to_npArray(test_dspl_path)
test_tracs = matFiles_to_npArray(test_trac_path)

Split training data into train and validation set using stratified samples.

In [63]:
radial_X_train, radial_X_val, radial_y_train, radial_y_val = train_test_split(dspl_radials, trac_radials, test_size=0.05, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(samples, targets, test_size=0.05)
X_train, X_val, y_train, y_val = np.append(radial_X_train, X_train), np.append(radial_X_val, X_val), np.append(radial_y_train, y_train), np.append(radial_y_val, y_val)

Drop (meta-) data which is not needed for training purposes.

In [64]:
X_train, X_val, y_train, y_val = np.array([sample['dspl'] for sample in X_train]), np.array([sample['dspl'] for sample in X_val]), np.array([target['trac'] for target in y_train]), np.array([target['trac'] for target in y_val])
X_test, y_test = np.array([sample['dspl'] for sample in test_dspls]), np.array([sample['trac'] for sample in test_tracs])

### Normalize data  !!! use some kind of max(max(x, y) to preserve the angle !!!

In [65]:
def normalize():
    return None

Extract displacement and traction fields from the data and reshape training set to (samples, channels, depth, heigth, width).

In [70]:
def reshape(array):
    """current shape is (samples, width, height, depth), reshape to (samples, channels, depth, height, width)"""
    return np.moveaxis(array[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

In [10]:
# Reshape to (samples, channels, depth, height, width)
X_train = np.moveaxis(X_train[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
X_val = np.moveaxis(X_val[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y_train = np.moveaxis(y_train[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y_val = np.moveaxis(y_val[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
X_test = np.moveaxis(X_test[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y_test = np.moveaxis(y_test[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

Convert train and validation data to Pytorch tensors.

In [11]:
X_train = torch.from_numpy(X_train).double()
X_val = torch.from_numpy(X_val).double()
y_train = torch.from_numpy(y_train).double()
y_val = torch.from_numpy(y_val).double()
X_test = torch.from_numpy(X_test).double()
y_test = torch.from_numpy(y_test).double()

In [12]:
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
test_set = TensorDataset(X_test, y_test)

batch_size = 4

dataloaders = {}
dataloaders['train'] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
dataloaders['val'] = DataLoader(val_set, batch_size=2*batch_size, num_workers=8, pin_memory=True)
dataloaders['test'] = DataLoader(test_set, batch_size=3*batch_size, shuffle=False, pin_memory=True)

## 2. Training

Sample inital weights for the convolutional layers from a normal distribution.

In [13]:
def initialize_weight(module):
    if isinstance(module, (nn.Conv3d, nn.ConvTranspose3d)):
        torch.nn.init.normal_(module.weight, std=0.01)

Define custom loss function corresponding to the forward loss function in the Matlab regression layer for image-to-image networks:
 
$${loss} = \frac{1}{2} \sum \limits _{p=1} ^{HWC} (t_{p} - y_{p})^{2}$$

In [14]:
class Custom_Loss(nn.Module):
    def __init__(self):
        super(Custom_Loss, self).__init__();
    
    def forward(self, predictions, target):
        loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
        return loss

In [15]:
def run_epoch(model, optimizer, dataloader, train):
    loss_fn = Custom_Loss()
    
    # Set model to training mode
    if train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0.0
    epoch_rmse = 0.0
    
    # Iterate over data
    for xb, yb in dataloader:
        xb, yb = xb.to(device), yb.to(device)

        # zero the parameters
        if train:
            optimizer.zero_grad()

        # forward
        with torch.set_grad_enabled(train):
            pred = model(xb)
            loss = loss_fn(pred, yb)

            # backward + optimize if in training phase
            if train:
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)                
                optimizer.step()

        # statistics
        epoch_loss += loss.item()
    
    epoch_loss /= len(dataloader.dataset)
    epoch_rmse = np.sqrt(2 * epoch_loss)
    return epoch_loss, epoch_rmse

In [16]:
def fit(model, optimizer, scheduler, dataloaders, max_epochs, patience):
    best_val_rmse = np.inf
    best_epoch = -1
    
    for epoch in range(1, max_epochs+1):
        train_loss, train_rmse = run_epoch(model, optimizer, dataloaders['train'], train=True)
        scheduler.step()
        val_loss, val_rmse = run_epoch(model, None, dataloaders['val'], train=False)
        print(f"Epoch {epoch}/{max_epochs}, train_loss: {train_loss:.3f}, train_rmse: {train_rmse:.3f}, val_loss: {val_loss:.3f}, val_rmse: {val_rmse:.3f}")
        
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('train_rmse', train_rmse, epoch)
        writer.add_scalar('val_loss', val_loss, epoch)
        writer.add_scalar('val_rmse', val_rmse, epoch)
        
        # Save best weights
        if val_rmse < best_val_rmse:
            best_epoch = epoch
            best_val_rmse = val_rmse
            best_model_weights = copy.deepcopy(model.state_dict())
            
        # Early stopping
        print(f"best val_rmse: {best_val_rmse:.3f}, epoch: {epoch}, best_epoch: {best_epoch}, current_patience: {patience - (epoch - best_epoch)}")
        if epoch - best_epoch >= patience:
            break
        
    torch.save(best_model_weights, f'/home/alexrichard/LRZ Sync+Share/ML in Physics/{NAME}.pth')

Instantiate the model (including logs for evaluation), the optimizer and start training.

In [17]:
NAME = "TracNet104-{:%Y-%b-%d %H:%M:%S}".format(datetime.datetime.now())
writer = SummaryWriter(log_dir='logs/{}'.format(NAME))
model = TracNet(n_channels=1).double()
model.to(device)
model.apply(initialize_weight)

inputs, targets = next(iter(dataloaders['train']))
inputs = inputs.cuda()
targets = targets.cuda()
writer.add_graph(model, inputs)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0006, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=10, gamma=0.7943, verbose=True)

fit(model, optimizer, scheduler, dataloaders, max_epochs=100, patience=5)

2022-06-02 16:54:19.368943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 16:54:19.368966: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/alexrichard/.local/lib/python3.8/site-packages/torch/nn/modules/conv.py:587: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:744.)
  return F.conv3d(


Adjusting learning rate of group 0 to 6.0000e-04.


KeyboardInterrupt: 

In [21]:
loss, rmse = test(model, Custom_Loss(), dataloaders['test'])

NameError: name 'test' is not defined

In [ ]:
loss

In [ ]:
rmse

Example predictions

In [ ]:
pred = model(X_test)

In [ ]:
pred[0][0]

In [ ]:
pred[0][0].shape

In [ ]:
pred_1 = (pred[0][0]).detach().numpy()

In [ ]:
pred_1 = np.moveaxis(pred_1, [0, 1, 2], [2, 1, 0])

In [ ]:
pred_1 = savemat('torch_pred_2.mat', {'trac': pred_1})

In [ ]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])

In [ ]:
matlab_predicted_trac_field.shape

In [ ]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])[np.newaxis, :]
matlab_predicted_trac_field = np.moveaxis(matlab_predicted_trac_field[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
matlab_predicted_trac_field = torch.from_numpy(matlab_predicted_trac_field).double()

In [ ]:
def forward(predictions, target):
    loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
    return loss

In [ ]:
forward(predicted_trac_field, gt_trac_field)

In [ ]:
forward(matlab_predicted_trac_field, gt_trac_field)

In [ ]:
torch.allclose(predicted_trac_field,matlab_predicted_trac_field, atol=5)

In [ ]:
predicted_trac_field

In [ ]:
matlab_predicted_trac_field